In [ ]:
#This file finds the best Discrete Wavelet transform to apply to the data and measurs average SNR
#Can be easily modified to save filtered data

In [1]:
import numpy as np
import matplotlib.pylab as plt
import padasip as pa
import torch
from scipy.io import wavfile
import os
import matplotlib.pylab as plt
import padasip as pa
import numpy as np
from scipy.io import wavfile
from scipy.fftpack import fft
from IPython.display import Audio
import IPython
import pywt
from skimage.restoration import denoise_wavelet


In [2]:
# creation of data
tests = ["Mic_on_Top-2/testing", "Mic_on_Top-2/training"]
commands = [
    "up",
    'go',
    "Noise"
]
labels = []
files = []

In [3]:
#put data into more useful datastructure
for path in tests:
    for file in os.listdir(path):
        command = file.split(".")[0]
        file_name = path+"/"+file
        files.append(file_name)
        labels.append(commands.index(command))

#Suplement with old noise data to make up for different quantities of data
for file in os.listdir("Recordings_Keyword_Noise/Noise/Medium"):
    command = "Noise"
    file_name = "Recordings_Keyword_Noise/Noise/Medium"+"/"+file
    files.append(file_name)
    labels.append(commands.index(command))

for file in os.listdir("Recordings_Keyword_Noise/Noise/Big"):
    command = "Noise"
    file_name = "Recordings_Keyword_Noise/Noise/Big"+"/"+file
    files.append(file_name)
    labels.append(commands.index(command))

recordedData = []
for i in range(len(labels)):
    recordedData.append((files[i], labels[i]))

In [4]:
#Set seeds for consistency
np.random.seed(0)
torch.manual_seed(0)

def zs(a):
    """ 1d data z-score """
    a -= a.mean()
    return a / a.std()

#snr Calculations
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)


FilteredDirectory = "./FilteredData/"

In [5]:
# constants
SAMPLERATE = 44100
n = 300 # filter size
D = 100 # signal delay
i = 0

#Create filter, it learns as we filter so we want to use the same one
np.random.shuffle(recordedData)
currentMaxSNR = 0
bestWav = None
for wav in pywt.wavelist():
    SNRs = []
    broke = False 
    for entry in recordedData:
        FILENAME = entry[0]


        # open and process source data
        fs, data = wavfile.read(FILENAME)
        y = data
        y = y.astype("float64")
        y = zs(y) / 10

        # contaminated with noise
        d = y 
        # apply filter
        try:
            e = denoise_wavelet(d, method= 'BayesShrink', mode= 'soft', wavelet_levels=3, wavelet= wav, rescale_sigma= 'True')
        except:
            broke = True
            break
        SNRs.append(signaltonoise(e))
        #Save file
        #saveFile = FilteredDirectory+commands[entry[1]]+"/"+commands[entry[1]]+str(i)+".wav"
        #print(saveFile)
        #wavfile.write(saveFile, 44100, e)
        i+=1
    if broke:
        continue
    print("Average SNR: ", sum(SNRs)/len(SNRs))
    if sum(SNRs)/len(SNRs) > currentMaxSNR:
        currentMaxSNR = sum(SNRs)/len(SNRs)
        bestWav = wav

print("bestWav: ", bestWav, "SNR: ", currentMaxSNR)



/opt/homebrew/Caskroom/miniforge/base/envs/ttmdrone/lib/python3.11/site-packages/skimage/restoration/_denoise.py:958: UserWarning: Wavelet thresholding was designed for use with orthogonal wavelets. For nonorthogonal wavelets such as bior1.1,results are likely to be suboptimal.
  out = _wavelet_threshold(image, wavelet=wavelet, method=method,
/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_58979/3914571844.py:19: WavFileWarning: Reached EOF prematurely; finished at 88244 bytes, expected 88252 bytes from header.
  fs, data = wavfile.read(FILENAME)
/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_58979/3914571844.py:19: WavFileWarning: Reached EOF prematurely; finished at 88246 bytes, expected 88254 bytes from header.
  fs, data = wavfile.read(FILENAME)
/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_58979/3914571844.py:19: WavFileWarning: Reached EOF prematurely; finished at 66780 bytes, expected 66788 bytes from header.
  fs, data = wavfile.read(FILENAME

Average SNR:  -2.006569598574712e-09


/opt/homebrew/Caskroom/miniforge/base/envs/ttmdrone/lib/python3.11/site-packages/skimage/restoration/_denoise.py:958: UserWarning: Wavelet thresholding was designed for use with orthogonal wavelets. For nonorthogonal wavelets such as bior1.3,results are likely to be suboptimal.
  out = _wavelet_threshold(image, wavelet=wavelet, method=method,


Average SNR:  1.5457985406743356e-08


/opt/homebrew/Caskroom/miniforge/base/envs/ttmdrone/lib/python3.11/site-packages/skimage/restoration/_denoise.py:958: UserWarning: Wavelet thresholding was designed for use with orthogonal wavelets. For nonorthogonal wavelets such as bior1.5,results are likely to be suboptimal.
  out = _wavelet_threshold(image, wavelet=wavelet, method=method,


...

Average SNR:  -1.6531277287624345e-07


/opt/homebrew/Caskroom/miniforge/base/envs/ttmdrone/lib/python3.11/site-packages/skimage/restoration/_denoise.py:958: UserWarning: Wavelet thresholding was designed for use with orthogonal wavelets. For nonorthogonal wavelets such as rbio5.5,results are likely to be suboptimal.
  out = _wavelet_threshold(image, wavelet=wavelet, method=method,


Average SNR:  1.998313140234707e-08


/opt/homebrew/Caskroom/miniforge/base/envs/ttmdrone/lib/python3.11/site-packages/skimage/restoration/_denoise.py:958: UserWarning: Wavelet thresholding was designed for use with orthogonal wavelets. For nonorthogonal wavelets such as rbio6.8,results are likely to be suboptimal.
  out = _wavelet_threshold(image, wavelet=wavelet, method=method,


Average SNR:  -4.005502571095383e-08
Average SNR:  4.056709727810983e-08
Average SNR:  1.4695686408298228e-07
Average SNR:  8.06028118740425e-09
Average SNR:  9.342349871555751e-08
Average SNR:  -6.99910291017277e-08
Average SNR:  -9.585567944264014e-08
Average SNR:  1.694406002417713e-07
Average SNR:  7.580520019301269e-08
Average SNR:  -3.8704867537780115e-08
Average SNR:  -8.868635719997106e-09
Average SNR:  4.32987449111614e-08
Average SNR:  -5.6679741128633344e-08
Average SNR:  3.2851335904915604e-08
Average SNR:  -1.1966545016372236e-07
Average SNR:  4.786212061982444e-08
Average SNR:  -5.958377770409618e-09
Average SNR:  -3.7577344782197714e-08
Average SNR:  -1.9680512669484273e-08
Average SNR:  -8.000768968574373e-08
bestWav:  rbio3.5 SNR:  2.9679592986413027e-07


In [8]:
# display sound players for human validation
print("Original record:")
IPython.display.display(Audio(y, rate=SAMPLERATE))

print("Enhanced record")
IPython.display.display(Audio(e, rate=SAMPLERATE))

print(FILENAME)


Original record:


Enhanced record


Mic_on_Top-2/training/go.1.wav.4k6m79cs.ingestion-6d6cc58874-2ngxd.s9.wav


In [9]:
print("Original SNR: ", signaltonoise(y))
print("Enhanced SNR: ", signaltonoise(e))

Original SNR:  -3.101575433873454e-17
Enhanced SNR:  -1.3239039988895115e-06
